In [1]:
from google.cloud import aiplatform

aiplatform.init(location='us-west1')

In [2]:
model_id = 'projects/ml-ops-segment-anything/locations/us-west1/models/7722696994614935552'
model = aiplatform.Model(model_id)

In [3]:
model.supported_input_storage_formats

['jsonl', 'bigquery', 'csv', 'tf-record', 'tf-record-gzip', 'file-list']

In [4]:
from google.cloud import storage
import base64
import json

# Initialize a client
storage_client = storage.Client()
# Create a bucket object
bucket = storage_client.get_bucket("samimages")
# Create blob objects from the filepath
blobs = bucket.list_blobs(prefix="images")
# Iterate over the blobs and filter based on file extension
image_extensions = ('.png', '.jpg', '.jpeg')
image_blobs = [blob for blob in blobs if blob.name.lower().endswith(image_extensions)]

# Download all images in String
image_base64_dict = {"instances": []}
for image_blob in image_blobs:
    base64_str = base64.b64encode(image_blob.download_as_bytes()).decode('utf-8')
    # image_base64_dict[image_blob.name] = base64_str
    image_base64_dict["instances"].append({
        'image': base64_str,
        'file_path': f"gs://samimages/{image_blob.name}"
    })

# Save it to Artifact
with open("batch_predict.jsonl", "w") as file:
    json.dump(image_base64_dict, file)
    
blob = bucket.blob("batch_predict.jsonl")
blob.upload_from_filename("batch_predict.jsonl")

In [5]:
job_display_name = 'sam'
gcs_source = 'gs://samimages/batch_predict.jsonl'
gcs_destination_prefix = 'gs://samimages'

In [6]:
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    machine_type= "n1-standard-2",
    accelerator_type= "NVIDIA_TESLA_T4",
    accelerator_count= 1,
    gcs_source=gcs_source,
    gcs_destination_prefix=gcs_destination_prefix,
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-west1/batch-predictions/3963534908969713664?project=633534855904
BatchPredictionJob projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/633534855904/locations/us-west1/batchPredictionJobs/3963534908969713664 c